<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import pandas as pd
import numpy as np
from PIL import Image
from pytorch_lightning import seed_everything, LightningModule, Trainer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import scipy.stats as stats
import pytorch_lightning as pl
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import nn, optim, rand, sum as tsum, reshape, save

In [38]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [39]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [41]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [42]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [43]:
train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")
validation = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/validation.csv")
test = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/boneage-test-dataset.csv")

In [44]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [45]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

In [46]:
train['boneage zscore'] = stats.zscore(train['boneage'])

In [47]:
data_train = pd.DataFrame(train)
data_train.rename(columns={'male':'gender'},inplace=True)

In [48]:
data_train

,id,boneage,gender,boneage zscore
0,1377.png,180,1,1.278370
1,1378.png,12,1,-2.796477
2,1379.png,94,1,-0.807564
3,1380.png,120,0,-0.176933
4,1381.png,82,1,-1.098624
...,...,...,...,...
10084,13259.png,94,1,-0.807564
10085,13261.png,94,1,-0.807564
10086,13274.png,94,1,-0.807564
10087,13286.png,94,1,-0.807564


In [49]:
train

,id,boneage,gender,boneage zscore
0,1377.png,180,1,1.278370
1,1378.png,12,1,-2.796477
2,1379.png,94,1,-0.807564
3,1380.png,120,0,-0.176933
4,1381.png,82,1,-1.098624
...,...,...,...,...
10084,13259.png,94,1,-0.807564
10085,13261.png,94,1,-0.807564
10086,13274.png,94,1,-0.807564
10087,13286.png,94,1,-0.807564


In [50]:
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

In [51]:
validation['boneage zscore'] = stats.zscore(validation['boneage'])

In [52]:
data_validation = pd.DataFrame(validation)
data_validation.rename(columns={'male':'gender'},inplace=True)

In [53]:
validation

,id,boneage,gender,boneage zscore
0,10186.png,10,0,-2.864934
1,10758.png,15,1,-2.742944
2,10860.png,180,1,1.282722
3,10877.png,180,1,1.282722
4,11008.png,180,1,1.282722
...,...,...,...,...
2517,15605.png,50,1,-1.889015
2518,15606.png,113,1,-0.351942
2519,15608.png,55,1,-1.767025
2520,15609.png,150,0,0.550783


In [54]:
test

,Case ID,Sex,boneage
0,4360,M,169
1,4361,M,170
2,4362,M,73
3,4363,M,153
4,4364,M,135
...,...,...,...
195,4555,F,133
196,4556,F,129
197,4557,F,168
198,4558,F,136


In [55]:
test['Case ID'] = test['Case ID'].apply(lambda x: str(x)+'.png')
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
#test['Sex'] = test['Sex'].apply(lambda x: 0 if x else 1)

data_test = pd.DataFrame(test)
data_test.rename(columns={'Sex':'gender'},inplace=True)
data_test['gender'].replace({ 'M':0,'F':1}, inplace=True)


In [56]:
test

,Case ID,gender,boneage
0,4360.png,0,169
1,4361.png,0,170
2,4362.png,0,73
3,4363.png,0,153
4,4364.png,0,135
...,...,...,...
195,4555.png,1,133
196,4556.png,1,129
197,4557.png,1,168
198,4558.png,1,136


#clone

##ลองดึง Inception_V3 จาก github ตัวเอง

In [57]:
#Local imports
#from Project_Boneage.backbone.inception_v3 import InceptionV3

##ลองดึง Inception_V3 จาก github ของ Bonet มาใช้ก่อน

In [58]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet


fatal: destination path 'Bonet' already exists and is not an empty directory.


In [59]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

#Creating a custom PyTorch Dataset

In [60]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, pickle_file, image_dir):
        self.image_dir = image_dir
        self.pickle_file = pickle_file
        self.tabular = pd.read_csv(pickle_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = tabular["boneage"]

        image = Image.open(f"{self.image_dir}{tabular['id']}.png")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        tabular = tabular[["male"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return image, tabular, y

In [61]:
table=pd.read_csv(path+"/train.csv")
table

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [62]:
np.array(Image.open(f"{train_img}{table.iloc[0,0:]['id']}.png"))[..., :3].shape

(2044, 3)

In [63]:
ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")

In [64]:
#import torchvision.models as models
#inception = models.inception_v3(init_weights=True)

In [65]:
class BoNetModule(pl.LightningModule):

    def __init__(self, transform_input = False, lr = 1e-3, num_workers = 0, batch_size = 2):
        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      image, tabular, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, tabular))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, 'correct': (y_pred.argmax(1) == y).type(torch.float), "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        train_loss = torch.stack([x['loss'] for x in outputs]).mean()
        train_accuracy = torch.cat([x['correct'] for x in outputs]).mean()
        self.log_dict({"loss": train_loss, "accuracy": train_accuracy})
        self.log_outputs["loss"] = train_loss
        self.log_outputs["accuracy"] = train_accuracy
        return

    def on_train_epoch_start(self) -> None:
        self.print(f"Epoch {self.trainer.current_epoch+1}/{self.trainer.max_epochs}")
        return super().on_train_epoch_start()

    def on_train_epoch_end(self) -> None:
        train_loss     = self.log_outputs["loss"]
        train_accuracy = self.log_outputs["accuracy"]
        valid_loss     = self.log_outputs["valid_loss"]
        valid_accuracy = self.log_outputs["valid_accuracy"]
        self.print(f"loss: {train_loss:.3f} - accuracy: {train_accuracy:.3f} - \
            val_loss: {valid_loss:.3f} - val_accuracy: {valid_accuracy:.3f}")
        return super().on_train_epoch_end()

    def on_train_start(self) -> None:
        self.print(f"Train start")
        return super().on_train_start()
        
    def on_train_end(self) -> None:
        self.print(f"Train end")
        return super().on_train_end()
 
    def validation_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss, 'val_correct': (y_pred.argmax(1) == y).type(torch.float)}

    def validation_epoch_end(self, outputs):
        valid_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        valid_accuracy = torch.cat([x['val_correct'] for x in outputs]).mean()
        self.log_dict({"valid_loss": valid_loss, "valid_accuracy": valid_accuracy})
        self.log_outputs["valid_loss"] = valid_loss
        self.log_outputs["valid_accuracy"] = valid_accuracy
        return

    def test_step(self, batch, batch_idx):
        image, tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, tabular))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")
        image_dataval = ImageDataset(pickle_file=path+"/validation.csv", image_dir=f"{val_img}")
        image_datatest = ImageDataset(pickle_file=path+"/boneage-test-dataset.csv", image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=(self.lr))
        return optimizer

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        logits = self.linear_relu_stack(x)
        return logits



In [66]:
ImageDataset(pickle_file=path+"/train.csv", image_dir=f"{train_img}")[0][0].size()

<ipython-input-60-4a3b2084aa90>:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  tabular = torch.FloatTensor(tabular)


torch.Size([1, 500, 500])

#Training

In [67]:
#from fastai.vision.learner import Learner
#from fastai.callback.schedule import lr_find

In [68]:
callbacks = EarlyStopping(monitor='valid_loss', patience=2)

In [69]:
checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss",
    dirpath="./pl-checkpoints",
    filename="sample-{epoch:02d}-{valid_loss:.03f}-{valid_accuracy:.03f}",
    save_top_k=3,
    mode="min",
)

In [70]:
model = BoNetModule()
trainer = pl.Trainer(
    max_epochs=2,
    callbacks=callbacks,
)
trainer.fit(model=model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type        | Params
---------------------------------------------
0 | inception_v3 | InceptionV3 | 21.8 M
1 | gender       | DenseLayer  | 64    
2 | fc_1         | DenseLayer  | 100 M 
3 | fc_2         | DenseLayer  | 1.0 M 
4 | fc_3         | Linear      | 1.0 K 
---------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     To

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
<ipython-input-60-4a3b2084aa90>:35: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  tabular = torch.FloatTensor(tabular)


AttributeError: ignored

In [ ]:
#logger = pl.loggers.TensorBoardLogger("lightning_logs",version = 'version_0', name="my_model")
#early_stop_callback = EarlyStopping(monitor='val_loss',min_delta=0.00,patience=2, verbose=True, mode='min')
#checkpoint_callback = ModelCheckpoint(verbose=True, mode='min')
#model = BoNetModule()
#trainer = Trainer(logger=logger,max_epochs = 2,callbacks=[checkpoint_callback])
#trainer.fit(model)
#print(trainer)
#%reload_ext tensorboard
#%tensorboard --logdir lightning_logs/

In [ ]:
#trainer = pl.Trainer(accelerator='cpu', logger=logger, callbacks=early_stop_callback)
#trainer = pl.Trainer(accelerator='cpu', gpus=1, logger=logger)#, early_stop_callback=early_stop_callback)

In [ ]:
# lr_finder = trainer.lr_find(model)
# fig = lr_finder.plot(suggest=True, show=True)
# new_lr = lr_finder.suggestion()
# print(new_lr)
# model.hparams.lr = new_lr

In [ ]:
#trainer.test(model)